In [1]:
require "hdf5"
require "optim"
require "nn"

In [2]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]
nfeatures = f:read("nfeatures"):all()[1]

--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

In [7]:
--Fits the count-based model with given smoothing parameters
--X : sequence features
--Y : sequence labels
--alpha1 : additive alpha for class counts
--alpha2 : additive alpha for class-conditional feature counts
function fit(X, Y, alpha1, alpha2)
    --count matrix of class transitions: p(y_i|y_{i-1},\theta)
    local C_trans = torch.ones(nclasses,nclasses)*alpha1
    --count matrix of class-conditional features: p(x_i|y_i,\theta)
    local C_emi = torch.ones(nclasses, nwords)*alpha2
    
    for i = 2,X:size(1) do 
        local y_curr = Y[i]
        local y_prev = Y[i-1]
        local x_curr = X[i]
        C_trans[y_prev][y_curr] = C_trans[y_prev][y_curr] + 1
        C_emi[y_curr][x_curr] = C_emi[y_curr][x_curr] + 1
    end
    C_trans:cdiv((C_trans:sum(2)):expand(C_trans:size(1),C_trans:size(2)))
    C_emi:cdiv((C_emi:sum(2)):expand(C_emi:size(1),C_emi:size(2)))
    return C_trans:log(), C_emi:log()
end

function score(y_prev, x_curr, C_trans, C_emi)
    local trans = C_trans[y_prev]
    local emi = C_emi[{{}, {x_curr, x_curr}}]
    return trans + emi
end

In [4]:
function score_hmm(observations, i)
    local observation_emission = emission[observations[i]]:view(C,1):expand(C,C)
    return observation_emission + transition
end

In [5]:
ct, ce = fit(X_train, Y_train, 1, 1)

     9
 13428
[torch.LongStorage of size 2]



In [8]:
score(2, 15, ct, ce)

 -6.0312
-10.5629
-17.6954
-17.6731
-16.9297
-17.5358
-17.5353
-13.3149
-17.5352
[torch.DoubleTensor of size 9]



In [31]:
X_valid_sen:size()[2]

63	


In [49]:
ct

Columns 1 to 8
 -0.2131  -3.3831  -3.2014  -3.9000  -3.9485 -10.6130 -10.6130  -2.5549
 -0.6095  -0.8523  -8.0301  -8.0301  -7.3369  -8.0301  -8.0301  -3.5874
 -0.1554  -7.7588  -2.1530  -6.1493  -5.3609  -7.7588  -7.0656  -3.9746
 -0.5409  -5.9885  -7.5979  -0.9233  -6.2116  -7.5979  -7.5979  -4.2657
 -0.3776  -4.0783  -6.0243  -4.6380  -1.3863  -4.9256  -7.1229  -3.6264
 -1.4469  -2.8332  -2.8332  -2.8332  -1.0415  -2.8332  -2.8332  -2.8332
 -1.6094  -2.3026  -2.3026  -2.3026  -2.3026  -2.3026  -2.3026  -2.3026
 -0.4625  -2.2543  -2.1091  -2.2374  -3.3221  -8.1179  -8.1179  -8.1179
 -2.1972  -2.1972  -2.1972  -2.1972  -2.1972  -2.1972  -2.1972  -2.1972

Columns 9 to 9
-10.6130
 -8.0301
 -7.7588
 -7.5979
 -7.1229
 -2.8332
 -2.3026
 -8.1179
 -2.1972
[torch.DoubleTensor of size 9x9]



In [20]:
ce:size()

     9
 13428
[torch.LongStorage of size 2]



In [ ]:
--Returns a distribution over the various tags
--y_prev : previous class tag
--x_curr : current feature 
--C_trans : transition count matrix
--C_emi : emission count matrix
function predict_distri(y_prev, x_curr, C_trans, C_emi)
    --compute transition and emission distributions
    local trans = C_trans[y_prev]/torch.sum(C_trans[y_prev])
    local emi = C_emi[{{},{x_curr,x_curr}}]/torch.sum(C_emi[{{},{x_curr,x_curr}}])
    return torch.log(trans) + torch.log(emi)
end

--Returns a log-probability table for a given single sentence and labels
--sen : a single sentence with word features
--labels : corresponding target labels for a sentence
--C_trans : transition count matrix
--C_emi : emission count matrix
function predict_table(sen, labels, C_trans, C_emi)
    local table = torch.ones(X_valid_sen:size()[2],nclasses)
    for j=1,sen:size()[1] do
        local feat = sen[j]
        --end of sentence
        if feat == 0 then
            break
        end
        table[j] = predict_distri(2, 15, C_trans, C_emi)
    end
    return table:t()
end

function viterbi(proba_table)
    
end